In [2]:
import pandas as pd

In [3]:
tr = pd.read_parquet('data/train.parquet')
tst = pd.read_parquet('data/test.parquet')

In [4]:
tst

,order_id,order_create_time,session_end_time,model_1,model_2,model_3,model_4,model_5
0,1350922,1976-02-05 15:08:37,1976-02-05 15:04:28.867,-1.710108,0.203183,0.040015,1,0.442343
1,1354989,1976-02-07 15:26:00,1976-02-07 15:27:26.593,0.086020,0.220081,0.021101,1,0.191027
2,1352637,1976-02-06 11:43:58,1976-02-05 18:34:59.477,-0.438741,0.644805,0.066312,0,0.400509
3,1350050,1976-02-05 11:26:19,1976-02-05 11:09:28.247,-1.395752,0.203183,0.083039,1,0.414590
4,1341733,1976-02-01 19:36:32,1976-02-01 09:53:54.103,-0.441443,0.644805,0.052902,0,0.411544
...,...,...,...,...,...,...,...,...
17191,1358397,1976-02-09 16:13:20,1976-02-02 09:02:14.140,-0.429658,0.644805,0.908250,0,0.196781
17192,1357968,1976-02-09 14:30:59,1976-02-09 14:26:24.480,-1.710108,0.203183,0.040665,0,0.411543
17193,1358835,1976-02-09 18:39:45,1976-02-09 18:30:36.407,0.086020,0.644805,0.039744,1,0.426308
17194,1365692,1976-02-12 17:50:18,1976-02-12 16:13:15.153,-1.710108,0.203183,0.041701,0,0.433750


In [4]:
target = tr.target
tr = tr.drop(columns='target')

order_id = tst.order_id
tst = tst.drop(columns='order_id')

In [5]:
import pandas as pd
import numpy as np

def create_time_features(df, order_time_col='order_create_time', session_end_col='session_end_time'):
    """
    Создает новые временные признаки на основе колонок с временными метками.
    
    Параметры:
    df - DataFrame с данными
    order_time_col - название колонки с временем создания заказа (timestamp)
    session_end_col - название колонки с временем окончания сессии (timestamp)
    
    Возвращает:
    DataFrame с добавленными новыми признаками
    """
    # Создаем копию DataFrame чтобы избежать предупреждений
    df = df.copy()
    
    # Преобразуем колонки в datetime, если они еще не в этом формате
    df[order_time_col] = pd.to_datetime(df[order_time_col])
    df[session_end_col] = pd.to_datetime(df[session_end_col])
    
    # 1. Разница между окончанием сессии и созданием заказа (в секундах)
    df['time_to_order_seconds'] = (df[session_end_col] - df[order_time_col]).dt.total_seconds()
    
    # 2. Признаки времени создания заказа
    df['order_hour'] = df[order_time_col].dt.hour
    df['order_day_of_week'] = df[order_time_col].dt.dayofweek  # 0-понедельник, 6-воскресенье
    df['order_day_of_month'] = df[order_time_col].dt.day
    df['order_month'] = df[order_time_col].dt.month
    df['order_week_of_year'] = df[order_time_col].dt.isocalendar().week
    df['order_is_weekend'] = df[order_time_col].dt.dayofweek >= 5
    
    # 3. Признаки времени окончания сессии
    df['session_end_hour'] = df[session_end_col].dt.hour
    df['session_end_day_of_week'] = df[session_end_col].dt.dayofweek
    df['session_end_day_of_month'] = df[session_end_col].dt.day
    df['session_end_month'] = df[session_end_col].dt.month
    df['session_end_week_of_year'] = df[session_end_col].dt.isocalendar().week
    df['session_end_is_weekend'] = df[session_end_col].dt.dayofweek >= 5
    
    # 4. Разница во времени в разных единицах
    df['time_to_order_minutes'] = df['time_to_order_seconds'] / 60
    df['time_to_order_hours'] = df['time_to_order_seconds'] / 3600
    
    # 5. Временные категории (утро/день/вечер/ночь)
    bins = [-1, 6, 12, 18, 23]
    labels = ['night', 'morning', 'afternoon', 'evening']
    df['order_time_of_day'] = pd.cut(df['order_hour'], bins=bins, labels=labels)
    df['session_end_time_of_day'] = pd.cut(df['session_end_hour'], bins=bins, labels=labels)
    
    # 6. Является ли заказ сделан в тот же день, что и сессия
    df['same_day_order'] = df[order_time_col].dt.date == df[session_end_col].dt.date
    
    # 7. Сезонность (времена года)
    def get_season(month):
        if month in [12, 1, 2]:
            return 'winter'
        elif month in [3, 4, 5]:
            return 'spring'
        elif month in [6, 7, 8]:
            return 'summer'
        else:
            return 'autumn'
    
    df['order_season'] = df['order_month'].apply(get_season)
    df['session_end_season'] = df['session_end_month'].apply(get_season)
    
    return df

In [6]:
tr = create_time_features(tr)
tr = tr.drop(columns=['order_create_time', 'session_end_time'])
tr

,model_1,model_2,model_3,model_4,model_5,time_to_order_seconds,order_hour,order_day_of_week,order_day_of_month,order_month,...,session_end_month,session_end_week_of_year,session_end_is_weekend,time_to_order_minutes,time_to_order_hours,order_time_of_day,session_end_time_of_day,same_day_order,order_season,session_end_season
0,0.798127,0.644805,0.039843,1,0.430339,-544.607,9,4,26,12,...,12.0,52,False,-9.076783,-0.151280,morning,morning,True,winter,winter
1,-0.573826,0.220081,0.020889,0,0.177593,1926.407,10,4,26,12,...,12.0,52,False,32.106783,0.535113,morning,morning,True,winter,winter
2,-0.665887,0.644805,0.039810,0,0.431897,-75.483,11,3,25,12,...,12.0,52,False,-1.258050,-0.020968,morning,morning,True,winter,winter
3,-0.284625,0.644805,0.040026,0,0.432045,831.153,14,4,26,12,...,12.0,52,False,13.852550,0.230876,afternoon,afternoon,True,winter,winter
4,-1.710108,-0.221541,0.020585,0,0.174115,-232.743,18,4,26,12,...,12.0,52,False,-3.879050,-0.064651,afternoon,afternoon,True,winter,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104590,-0.784541,0.220081,0.020901,0,0.188639,2674.990,12,1,16,12,...,12.0,51,False,44.583167,0.743053,morning,morning,True,winter,winter
104591,-0.504846,0.220081,0.029831,0,0.175063,-41689.227,11,5,8,11,...,11.0,45,False,-694.820450,-11.580341,morning,evening,False,autumn,autumn
104592,-1.710108,-0.221541,0.021167,0,0.191408,51.047,9,2,12,11,...,11.0,46,False,0.850783,0.014180,morning,morning,True,autumn,autumn
104593,-0.403279,0.644805,0.040814,0,0.452594,-304.027,17,0,10,11,...,11.0,46,False,-5.067117,-0.084452,afternoon,afternoon,True,autumn,autumn


In [7]:
tst = create_time_features(tst)
tst = tst.drop(columns=['order_create_time', 'session_end_time'])
tst

,model_1,model_2,model_3,model_4,model_5,time_to_order_seconds,order_hour,order_day_of_week,order_day_of_month,order_month,...,session_end_month,session_end_week_of_year,session_end_is_weekend,time_to_order_minutes,time_to_order_hours,order_time_of_day,session_end_time_of_day,same_day_order,order_season,session_end_season
0,-1.710108,0.203183,0.040015,1,0.442343,-248.133,15,3,5,2,...,2.0,6,False,-4.135550,-0.068926,afternoon,afternoon,True,winter,winter
1,0.086020,0.220081,0.021101,1,0.191027,86.593,15,5,7,2,...,2.0,6,True,1.443217,0.024054,afternoon,afternoon,True,winter,winter
2,-0.438741,0.644805,0.066312,0,0.400509,-61738.523,11,4,6,2,...,2.0,6,False,-1028.975383,-17.149590,morning,afternoon,False,winter,winter
3,-1.395752,0.203183,0.083039,1,0.414590,-1010.753,11,3,5,2,...,2.0,6,False,-16.845883,-0.280765,morning,morning,True,winter,winter
4,-0.441443,0.644805,0.052902,0,0.411544,-34957.897,19,6,1,2,...,2.0,5,True,-582.631617,-9.710527,evening,morning,True,winter,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17191,-0.429658,0.644805,0.908250,0,0.196781,-630665.860,16,0,9,2,...,2.0,6,False,-10511.097667,-175.184961,afternoon,morning,False,winter,winter
17192,-1.710108,0.203183,0.040665,0,0.411543,-274.520,14,0,9,2,...,2.0,7,False,-4.575333,-0.076256,afternoon,afternoon,True,winter,winter
17193,0.086020,0.644805,0.039744,1,0.426308,-548.593,18,0,9,2,...,2.0,7,False,-9.143217,-0.152387,afternoon,afternoon,True,winter,winter
17194,-1.710108,0.203183,0.041701,0,0.433750,-5822.847,17,3,12,2,...,2.0,7,False,-97.047450,-1.617457,afternoon,afternoon,True,winter,winter


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

categorical_features = [
    'order_time_of_day', 
    'session_end_time_of_day',
    'order_season',
    'session_end_season',
    'order_is_weekend',
    'session_end_is_weekend',
    'same_day_order'
]

for col in ['order_season', 'session_end_season']:
    tr[col] = tr[col].astype('category')
    tst[col] = tst[col].astype('category')

'''for feature in categorical_features:
    if feature in tr.columns:
        le = LabelEncoder()
        # Объединяем train и test для консистентного кодирования
        combined = pd.concat([tr[feature], tst[feature]], axis=0)
        le.fit(combined.astype(str))
        tr[feature] = le.transform(tr[feature].astype(str))
        tst[feature] = le.transform(tst[feature].astype(str))
        
        # Указываем LightGBM, что это категориальные признаки
        tr[feature] = tr[feature].astype('category')
        tst[feature] = tst[feature].astype('category')'''

# Разделим train на train/valid
X_train, X_valid, y_train, y_valid = train_test_split(
    tr, target, test_size=0.2, random_state=42, stratify=target
)

# Создадим датасеты для LightGBM
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data, categorical_feature=categorical_features)

# Параметры модели (можно настроить)
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_child_samples': 20,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'seed': 42,
    'early_stopping_round': 50
}

# Обучение модели
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, valid_data],
    valid_names=['train', 'valid'],
    num_boost_round=1000,
    callbacks=[
		lgb.early_stopping(stopping_rounds=50, verbose=True),
		lgb.log_evaluation(period=50)
	]
)

# Предсказание на валидации
val_pred = model.predict(X_valid)
val_auc = roc_auc_score(y_valid, val_pred)
print(f'Validation AUC: {val_auc:.4f}')

# Предсказание на тесте
test_pred = model.predict(tst)

# Сохранение результатов
submission = pd.DataFrame({
    'order_id': order_id,
    'target': test_pred
})

submission.to_csv('fuck_this_shit.csv', index=False)

Training until validation scores don't improve for 50 rounds
[50]	train's auc: 0.957483	valid's auc: 0.954893
[100]	train's auc: 0.960324	valid's auc: 0.956006
[150]	train's auc: 0.962937	valid's auc: 0.956309
[200]	train's auc: 0.965255	valid's auc: 0.956635
[250]	train's auc: 0.966934	valid's auc: 0.956623
Early stopping, best iteration is:
[207]	train's auc: 0.965457	valid's auc: 0.956672
Validation AUC: 0.9567


In [ ]:
'''import pickle

with open('models/lgb_944.pkl', 'wb') as f:
    pickle.dump(model, f)'''